In [ ]:
from collections import defaultdict
import math
import pandas as pd
import re
from datetime import datetime 
import numpy as np
from decimal import Decimal
print('numpy: ',np.version.version)
print('pandas: ',pd.__version__)
languages = ['en', 'es', 'eu', 'ca', 'pt', 'gl']

In [ ]:
df = pd.read_csv('../OriginalDataSet/training-tweets.txt', encoding='utf-8', error_bad_lines=False, sep='\t', nrows=15000)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]

In [ ]:
_df0 = df[['Language', 'Tweet']].copy()
pattern = re.compile('[ ]')
_df0.head()

In [ ]:
for language in languages:
    exec("%sAlphabets={}" % (language))
    exec ('{0}Size = 0'.format(language))

In [ ]:
start_time = datetime.now() 
trainDict = defaultdict(list)
for index, row in _df0.iterrows():
    sentence = ''
    tweet = row['Tweet']
    language = row['Language']
    for letter in tweet:
        if letter.isalpha() or pattern.match(letter):
            exec('if \'{0}\' not in {1}Alphabets.keys():\n\
                     {2}Alphabets[letter] = {3}Size\n\
                     {4}Size += 1'.format(letter, language, language, language, language))
            sentence = sentence + letter
    trainDict[row['Language']].append(sentence)
print('Reading time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [ ]:
print(enSize)
print(enAlphabets)
print(esSize)
print(esAlphabets)
print(euSize)
print(euAlphabets)
print(caSize)
print(caAlphabets)
print(ptSize)
print(ptAlphabets)
print(glSize)
print(glAlphabets)

In [ ]:
for language in languages:
    exec("%sModel=np.zeros(shape=((%sSize+1),(%sSize+1))) " %(language, language, language))
#     exec("%sCount=0" %(language))
#     exec("%sEachWordCount=dict.fromkeys(%sAlphabets, 0)" %(language, language)) #needed for bigrams

In [ ]:
print('EN',enModel, enModel.shape)
print('ES',esModel, esModel.shape)
print('EU',euModel, euModel.shape)
print('CA',caModel, caModel.shape)
print('PT',ptModel, ptModel.shape)
print('gl',glModel, glModel.shape)

In [ ]:
# a = dict.fromkeys(a, 0)

#### For Unigrams

In [ ]:
# start_time = datetime.now()
# delta = 0.5
# for language, tweets in trainDict.items():
#     for tweet in tweets:
#         for i in range(len(tweet)-1):
#             first = tweet[i]
#             exec('index = %sAlphabets[first]'%(language)) #index = esAlphabets[first] #get index of the character from the language dictionary
#             exec('%sModel[index] += 1'%(language)) #np.add.at(esModel, [index], 1) #increment that index in the language model
# for language in languages:
#     exec('%sModel = %sModel/%sModel.sum()'%(language, language, language)) #divide all the values by the sum of the row
#     exec('%sModel = np.log10(%sModel)'%(language, language))
# print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

#### For Bigrams

In [ ]:
start_time = datetime.now()
delta = 0.5
for language, tweets in trainDict.items():
    for tweet in tweets:
        for i in range(len(tweet)-2):
            first = tweet[i]
            second = tweet[i+1]
            exec('firstIndex = %sAlphabets[first]'%(language))
            exec('secondIndex = %sAlphabets[second]'%(language))
            exec('%sModel[firstIndex][secondIndex] += 1'%(language))
for language in languages:
    exec('%sModel = np.add(%sModel, delta)'%(language, language)) #this is where smoothing happens
    exec('%sModel = %sModel/%sModel.sum(axis=1)'%(language, language, language)) #divide all the values by the sum of the row
    exec('%sModel = np.log10(%sModel)'%(language, language))
print('Training time (hh:mm:ss.ms) {}'.format(datetime.now() - start_time ))

In [ ]:
print('EN',enModel, enModel.shape)
print('ES',esModel, esModel.shape)
print('EU',euModel, euModel.shape)
print('CA',caModel, caModel.shape)
print('PT',ptModel, ptModel.shape)
print('gl',glModel, glModel.shape)

In [ ]:
# glSum = glModel.sum()
# ptSum = ptModel.sum()
# caSum = caModel.sum()
# euSum = euModel.sum()
# esSum = esModel.sum()
# enSum = enModel.sum()

In [ ]:
# print('enSum', enSum ,'\nesSum', esSum ,'\neuSum', euSum ,'\ncaSum', caSum ,'\nptSum', ptSum, '\nglSum', glSum)

## Testing the models

In [ ]:
df = pd.read_csv('../OriginalDataSet/test-tweets-given.txt', encoding='utf-8', error_bad_lines=False, sep='\t', nrows=100)
df.columns = ['TweetID', 'UserID', 'Language', "Tweet"]
_df0 = df[['TweetID', 'Language', 'Tweet']].copy()
# pattern = re.compile('[a-z ]')
_df0.head()

In [ ]:
probability = {}
for index, row in _df0.iterrows():
    for language in languages:
        exec("%sProb=math.log10(1/6)" % (language))
    tweetID = row['TweetID']
    langTweet = row['Language']
    tweet = row['Tweet']
    
    for i in range(len(tweet)-2):
        first = tweet[i]
        second = tweet[i+1]
        for lang in languages:
            exec('if ((first not in {0}Alphabets.keys()) and (second not in {1}Alphabets.keys())):\n\
    prob = {2}Model[-1][-1]\n\
elif (second not in {3}Alphabets.keys()):\n\
    index = {4}Alphabets[first]\n\
    prob = {5}Model[index][-1]\n\
elif (first not in {6}Alphabets.keys()):\n\
    index = {7}Alphabets[second]\n\
    prob = {8}Model[-1][index]\n\
else:\n\
    firstIndex = {9}Alphabets[first]\n\
    secondIndex = {10}Alphabets[second]\n\
    prob = {11}Model[firstIndex][secondIndex]\n\
{12}Prob = prob + {13}Prob\n'.format(lang, lang, lang, lang, lang, lang, lang, lang, lang, lang, lang, lang, lang, lang))
    for langu in languages:
        exec("probability['%s'] = %sProb"%(langu, langu))
#     probability[lang] = enProb
#     print(probability)
#     print('tweet: ', tweet,'\nenProb:', enProb, '\neuProb:', euProb, '\nesProb:', esProb, '\ncaProb:', caProb
#     ,'\nptProb:', ptProb, '\nglProb:', glProb)
    result = max(probability, key=probability.get)
    print(tweetID, '  ', result, '  ', '%.2E' % Decimal(probability[result]), '  ', langTweet, 'correct' if (langTweet==result) else 'wrong')

In [ ]:
# probability = {}
# for index, row in _df0.iterrows():
#     for language in languages:
#         exec("%sProb=math.log10(1/6)" % (language))
#     tweetID = row['TweetID']
#     lang = row['Language']
#     tweet = row['Tweet']
#     for i in range(len(tweet)-2):
#         first = tweet[i]
#         second = tweet[i+1]
#         for lang in languages:
#             if ((first not in enDict.keys()) and (second not in enDict.keys())):
#                 #take the last entry of the model
#                 prob = enModel[-1][-1]
#             elif (second not in enDict.keys()):
#                 #take the coreesponding row but last column
#                 index = enDict[first]
#                 prob = enModel[index][-1]
#             elif (first not in enDict.keys()): 
#                 #take the last row and the corresponding column
#                 index = enDict[second]
#                 prob = enModel[-1][index]
#             else:
#                 #both are in the matrix
#                 firstIndex = enDict[first]
#                 secondIndex = enDict[second]
#                 prob = enModel[firstIndex][secondIndex]
#             enProb = prob + enProb
#     probability['en'] = enProb
    
#     result = max(probability, key=probability.get)
#     print(tweetID, '  ', result, '  ', '%.2E' % Decimal(probability[result]), '  ', lang, 'correct' if (lang==result) else 'wrong')